## 00 - Preparamos el entorno cargando las bibliotecas necesarias
En caso de que dichas bibliotecas no estén instaladas, es necesario ejecutar el siguiente bloque de código
```
! pip install --upgrade requests numpy pandas
```

In [ ]:
import requests
import numpy as np
import json
import pandas as pd

## 01 - Preparación de la solicitud
Cuando interactuar con el servidor [www.transparenciapresupuestaria.gob.mx](https://www.transparenciapresupuestaria.gob.mx), dicho servidor espera información por parte nuestra.

En el navegador, el frontend se encarga de preparar el request mediante la interacción con el usuario

<img src="transparenciapresupuestaria.png" width="500px"/>

El servidor remoto usa la api albergada en [https://nptp.hacienda.gob.mx/programas/mapa](https://nptp.hacienda.gob.mx/programas/mapa).

Qué pasa si haces click en el link anterior?

### 01.1 - Un diccionario simple
En la siguiente celda, podrás construir un diccionario en el que se almacenan los distintos parámetros que le podemos pasar a la API de Transparencia Presupuestaria

In [ ]:
url_data = {"id_entidad_federativa":"11",
            "id_ramo":"",
            "desc_ppi":"",
            "id_clave_cartera":"",
            "page":"1",
            "pageSize":"20"}

### 01.2 - Limpieza de datos

Para un humano es sencillo notar si algo está vacío, sencillamente lo omites y listo.

Para una API esto no siempre es obvio, por lo que antes de lanzar nuestra solicitud, debemos limpiar el diccionario, para no pasarle parámetros vacíos a la solicitud

In [ ]:
remove_keys = []
for key in url_data.keys():
    if (url_data[key] == ""):
        remove_keys.append(key)
for key in remove_keys:
    del url_data[key]

In [ ]:
print(url_data)

## 02 - Solicitud usando requests

Ya que limpiamos el set de parámetros que le vamos a pasar a la URL de la API, podemos lanzar la solicitud

No obstante, si el set de parámetros está vacío, mejor es no pasarle nada a la API y usar los defaults

In [ ]:
if(len(url_data.keys())==0):
    url_request = requests.get('https://nptp.hacienda.gob.mx/programas/mapa')
else:
    url_request = requests.get('https://nptp.hacienda.gob.mx/programas/mapa',params=url_data)

### 02.1 Safety checks

Si todo salió bien con el código anterior, podemos revisar algunos elementos de nuestra solicitud

Al ejecutar el código en la siguiente celda podemos ver el URL de la API.

Puedes hacer click en el link y revisar el output en el navegador

>**Feeling daring?**

>Tambien puedes usar [curl](https://curl.se/) y [JQ](https://jqlang.github.io/jq/) para llamar a la API y obtener resultados de consulta en tu terminal

>La regla no escrita en cómputo es que siempre hay más de una manera de hacer las cosas

In [ ]:
print(url_request.url)

### 02.2 Status code

Si bien realizamos la solicitud exitosamente, no significa que el servidor haya aceptado dicha solicitud

Podemos checar el status code como una instancia de requests

Y en una sola línea, podemos compararlo con códigos permisibles

In [ ]:
if (url_request.status_code == requests.codes.ok):
    print("Solicitud efectuada exitosamente")
else:
    print("Error en la solicitud"+str(url_request.status_code))

In [ ]:
url_request.status_code

## 03 - Data retrieval

Si nuestra solicitud se llevó a cabo exitosamente, el servidor nos entregará la información en formato [JSON](https://en.wikipedia.org/wiki/JSON)

>***Es sumamente importante revisar la API de cada sitio web, aún hay sitios que entregan XML, texto plano o incluso HTML***

>Los parseadores que usemos para obtener información de cada sitio deberán ajustarse al formato que el servidor entreguen

In [ ]:
if (url_request.status_code == requests.codes.ok):
    retrieved_data = url_request.json()

In [ ]:
retrieved_data

### 03.1 Resumen de nuestros datos

Antes de procesar los datos es indispensable tener una idea de:

- Qué tamaño tienen
- Cómo están estructurados
- Qué tipo de datos están almacenados en cada *campo*

#### 03.1.1 Tamaño
JSON es un formato que puede ser tan superficial o tan profundo cómo queramos, con el siguiente código podemos ver cuantos elementos nos mandó el servidor a nivel superficial

In [ ]:
print(len(retrieved_data))

#### 03.1.2 Estructura

En las siguientes celdas podemos obtener información relevante acerca de la estructura de la información obtenida del servidor remoto

In [ ]:
type(retrieved_data)

In [ ]:
retrieved_keys = retrieved_data.keys()
print(retrieved_keys)

#### 03.1.3 Tipo de datos

La información obtenida está organizada en 5 elementos superficiales, sin embargo, uno de ellos tiene más de lo que aparenta

In [ ]:
for key in retrieved_keys:
    print(key + "\t" +str(type(retrieved_data[key])))

#### 03.1.4 Estructura, again

El elemento "results" dentro de nuestro objeto de python es de tipo `list`

Ésto nos indica que podemos iterar sobre dicha lista y accesar los valores presentes en ella

In [ ]:
print(len(retrieved_data["results"]))

In [ ]:
print(type(retrieved_data["results"][0]))

In [ ]:
print(retrieved_data["results"][0].keys())

In [ ]:
results_keys = retrieved_data["results"][0].keys()
for key in results_keys:
    print(key + "\t" +str(type(retrieved_data["results"][0][key])))

## 04 - Obteniendo información básica

### 04.1 Elementos superficiales

Los elementos superficiales del diccionario `retrieved_data` los podemos accesar directamente a través del nombre de su llave

In [ ]:
entries   = retrieved_data["total"]
page      = retrieved_data["page"]
pages     = retrieved_data["pages"]
page_size = retrieved_data["pageSize"]

### 04.2 Elementos anidados

Los elementos anidados, como `results`, tenemos que sub indizarlos, es decir, primero llamamos la llave results y luego el índice de cada sub elemento

Para accesar al primer elemento llamamos al índice `[0]`

In [ ]:
retrieved_data["results"][0]

In [ ]:
type(retrieved_data["results"][0])

## 05 - Nuestro mejor amigo el diccionario

Los desarrolladores detrás de las APIs usualmente muestran los datos de forma bien estructurada.

Podemos tomar ventaja de ello y convertir todo el elemento `results` en un dataframe usando pandas

In [ ]:
df = pd.DataFrame.from_dict(retrieved_data["results"])

In [ ]:
df

### 05.1 Ahorrandonos unos clicks

Ya que tenemos nuestro dataframe, podemos exportarlo a algo más legible y transportable

In [ ]:
df.to_csv("resultados.tsv",sep="\t",index=False)

## 06 - Entrando al agujero de conejo

En el sitio de transparencia presupuestaria puedes consultar más información acerca de cada proyecto

El sitio te lleva a la página [https://www.transparenciapresupuestaria.gob.mx/ficha_opa](https://www.transparenciapresupuestaria.gob.mx/ficha_opa)

Sin embargo, en dicha página no puedes hacer consultas si no es desde la página anterior (Obra Pública Abierta)

Si analizamos nuestro entorno de desarrollo en el navegador, veremos que se hace una llamada a una segunda API

Dicha API está albergada en [https://nptp.hacienda.gob.mx/programas/detalle](https://nptp.hacienda.gob.mx/programas/detalle)

Y espera como único parámetro `id_clave_cartera`

### 06.1 Preparando el acceso por lote

Ya sea desde nuestro dataframe o desde nuestro objeto JSON, podemos obtener una lista de los valores de `id_clave_cartera`

Con dicha lista podemos interrogar la segunda API para obtener información más detallada de cada proyecto

>Spoiler, encontraremos más información de la que se muestra en el sitio web!

Primero construimos una lista y la llenamos con los valores de `id_clave_cartera`

In [ ]:
id_clave_cartera_list = []
for entry in range(page_size):
    id_clave_cartera_list.append(retrieved_data["results"][entry]["id_clave_cartera"])

In [ ]:
id_clave_cartera_list

### 06.2 Una prueba rápida

Antes de lanzar toda la lista de ids, probemos con solamente un elemento.

Haremos lo mismo que en la primera parte, prepararemos un diccionario que contiene los parámetros que le pasaremos a la API

In [ ]:
ext_data = {"id_clave_cartera" : "0616B000034"}

#### 06.2.1 Igualmente haremos una limpieza de datos

In [ ]:
remove_keys = []
for key in ext_data.keys():
    if (ext_data[key] == ""):
        remove_keys.append(key)
for key in remove_keys:
    del ext_data[key]
if(len(ext_data.keys())!=0):
    ext_request = requests.get("https://nptp.hacienda.gob.mx/programas/detalle",params=ext_data)

In [ ]:
print(ext_request.url)

#### 06.2.2 Status code

In [ ]:
if (ext_request.status_code == requests.codes.ok):
    print("Solicitud efectuada exitosamente")
else:
    print("Error en la solicitud"+str(ext_request.status_code))

In [ ]:
if (ext_request.status_code == requests.codes.ok):
    retrieved_details = ext_request.json()

#### 06.2.3 Exploración de los datos

En este punto ya es más sencillo ir directo sobre los datos

Vamos a analizar el objeto recibido y pensemos en formas de transformar la información recibida en formatos más manejables

Pensemos tambien en cuales serían los siguientes pasos para la automatización de extracción de información, tanto de la primera como de la segunda API

In [ ]:
retrieved_details

In [ ]:
for key in retrieved_details[0].keys():
    print(key+"\t"+str(type(retrieved_details[0][key])))

In [ ]:
useful_keys = []

In [ ]:
page_size = 20
url_data = {"id_entidad_federativa":"11",
            "page":"1",
            "pageSize":page_size}
url_request = requests.get('https://nptp.hacienda.gob.mx/programas/mapa',params=url_data)
retrieved_data = url_request.json()
total_results = retrieved_data["total"]
num_queries = int(total_results/page_size) + 1 
for query in range(num_queries):
    query_data = {"id_entidad_federativa":"11",
                  "page":query,
                  "pageSize":page_size}
    query_request = requests.get('https://nptp.hacienda.gob.mx/programas/mapa',params=query_data)
    query_retrieved_data = query_request.json()